# YOLOv8-Nano Detection Model Training
## Cattle Breed Recognition System - Stage 1: Animal Detection

This notebook trains a YOLOv8-Nano model for detecting cattle and buffaloes in images.

**Model Specifications:**
- Model: YOLOv8-Nano (smallest, fastest)
- Input Size: 416x416
- Output: Bounding box (x, y, w, h) for animal detection
- Final Size: ~5 MB (after INT8 quantization)
- Inference: ~20ms on mobile

**Author:** SIH 2025 Team
**Problem Statement:** SIH25004 - Image-based Breed Recognition for Cattle and Buffaloes of India

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

# Install YOLOv8
!pip install ultralytics==8.0.196 -q
!pip install roboflow -q

In [ ]:
import os
import yaml
import json
import shutil
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
from google.colab import drive

# Mount Google Drive for data storage
drive.mount('/content/drive')

# Set paths
BASE_PATH = '/content/drive/MyDrive/cattle_breed_recognition'
DATA_PATH = f'{BASE_PATH}/data'
MODELS_PATH = f'{BASE_PATH}/models'
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)

print(f"Base Path: {BASE_PATH}")
print(f"Data Path: {DATA_PATH}")
print(f"Models Path: {MODELS_PATH}")

## 2. Prepare Dataset

### Option A: Download from Roboflow (Recommended)

In [ ]:
# Option A: Download cattle detection dataset from Roboflow
# You can find cattle detection datasets at: https://universe.roboflow.com/

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("workspace-name").project("cattle-detection")
dataset = project.version(1).download("yolov8")

# The dataset will be downloaded in YOLOv8 format
DATASET_PATH = dataset.location
print(f"Dataset downloaded to: {DATASET_PATH}")

### Option B: Use Local Dataset

In [ ]:
# Option B: Upload your own dataset
# Dataset structure should be:
# dataset/
# ├── data.yaml
# ├── train/
# │   ├── images/
# │   └── labels/
# ├── valid/
# │   ├── images/
# │   └── labels/
# └── test/
#     ├── images/
#     └── labels/

# Upload dataset.zip to Colab and extract
# !unzip dataset.zip -d {DATA_PATH}

# DATASET_PATH = f'{DATA_PATH}/dataset'
# print(f"Dataset path: {DATASET_PATH}")

### Option C: Create Dataset from Kaggle Images

In [ ]:
# Option C: Create detection dataset from Kaggle breed images
# This creates bounding boxes around the entire image (simple approach)

def create_detection_dataset_from_classification(source_dir, output_dir, class_name='cattle'):
    """
    Create YOLO format detection dataset from classification images.
    Uses entire image as bounding box.
    
    Args:
        source_dir: Directory with class folders (classification format)
        output_dir: Output directory for YOLO format
        class_name: Class name for detection (cattle/buffalo)
    """
    import random
    
    # Create output directories
    for split in ['train', 'valid', 'test']:
        os.makedirs(f'{output_dir}/{split}/images', exist_ok=True)
        os.makedirs(f'{output_dir}/{split}/labels', exist_ok=True)
    
    # Collect all images
    all_images = []
    for breed_folder in os.listdir(source_dir):
        breed_path = os.path.join(source_dir, breed_folder)
        if os.path.isdir(breed_path):
            for img_name in os.listdir(breed_path):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    all_images.append({
                        'path': os.path.join(breed_path, img_name),
                        'name': f"{breed_folder}_{img_name}"
                    })
    
    print(f"Total images found: {len(all_images)}")
    
    # Shuffle and split
    random.shuffle(all_images)
    n = len(all_images)
    train_split = int(0.7 * n)
    val_split = int(0.85 * n)
    
    splits = {
        'train': all_images[:train_split],
        'valid': all_images[train_split:val_split],
        'test': all_images[val_split:]
    }
    
    # Process each split
    for split_name, images in splits.items():
        print(f"Processing {split_name}: {len(images)} images")
        
        for img_info in images:
            # Copy image
            img_path = img_info['path']
            img_name = img_info['name']
            dest_img = f'{output_dir}/{split_name}/images/{img_name}'
            shutil.copy(img_path, dest_img)
            
            # Get image dimensions
            img = cv2.imread(img_path)
            h, w = img.shape[:2]
            
            # Create YOLO format label (entire image as bbox)
            # YOLO format: class x_center y_center width height (normalized 0-1)
            # class 0 = cattle
            label_name = img_name.rsplit('.', 1)[0] + '.txt'
            label_path = f'{output_dir}/{split_name}/labels/{label_name}'
            
            # Entire image as bounding box
            x_center = 0.5
            y_center = 0.5
            width = 1.0
            height = 1.0
            
            with open(label_path, 'w') as f:
                f.write(f"0 {x_center} {y_center} {width} {height}\n")
    
    print(f"Dataset created at: {output_dir}")
    return output_dir

# Uncomment to use:
# SOURCE_DATA = f'{DATA_PATH}/raw/kaggle_cattle_images'
# OUTPUT_DATA = f'{DATA_PATH}/detection_dataset'
# create_detection_dataset_from_classification(SOURCE_DATA, OUTPUT_DATA)

## 3. Create data.yaml Configuration

In [ ]:
# Create data.yaml for YOLOv8

def create_data_yaml(dataset_path, output_path):
    """
    Create data.yaml configuration file for YOLOv8 training.
    """
    data_config = {
        'path': dataset_path,
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'nc': 1,  # Number of classes (1 = cattle/buffalo)
        'names': ['cattle']  # Class names
    }
    
    with open(output_path, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)
    
    print(f"Created data.yaml at: {output_path}")
    return output_path

# Create data.yaml
DATASET_PATH = f'{DATA_PATH}/detection_dataset'  # Update this path
YAML_PATH = create_data_yaml(DATASET_PATH, f'{DATASET_PATH}/data.yaml')

# Display config
with open(YAML_PATH, 'r') as f:
    print(f.read())

## 4. Train YOLOv8-Nano Model

In [ ]:
# Load YOLOv8-Nano model (pretrained on COCO)
model = YOLO('yolov8n.pt')  # Nano model

# Training parameters
training_params = {
    'data': YAML_PATH,
    'epochs': 100,              # Number of training epochs
    'imgsz': 416,               # Image size (smaller for mobile)
    'batch': 32,                # Batch size
    'name': 'cattle_detector',  # Experiment name
    'project': MODELS_PATH,     # Save directory
    'device': 0,                # GPU device
    'patience': 20,             # Early stopping patience
    'save': True,               # Save checkpoints
    'save_period': 10,          # Save every N epochs
    'workers': 4,               # Data loading workers
    'pretrained': True,         # Use pretrained weights
    'optimizer': 'auto',        # Optimizer (auto selects AdamW)
    'lr0': 0.01,                # Initial learning rate
    'lrf': 0.01,                # Final learning rate
    'momentum': 0.937,          # SGD momentum
    'weight_decay': 0.0005,     # Weight decay
    'warmup_epochs': 3,         # Warmup epochs
    'warmup_momentum': 0.8,     # Warmup momentum
    'warmup_bias_lr': 0.1,      # Warmup bias learning rate
    'box': 7.5,                 # Box loss gain
    'cls': 0.5,                 # Classification loss gain
    'dfl': 1.5,                 # Distribution focal loss gain
    'pose': 12.0,               # Pose loss gain
    'kobj': 1.0,                # Keypoint objectness loss gain
    'label_smoothing': 0.0,     # Label smoothing
    'nbs': 64,                  # Nominal batch size
    'hsv_h': 0.015,             # HSV-Hue augmentation
    'hsv_s': 0.7,               # HSV-Saturation augmentation
    'hsv_v': 0.4,               # HSV-Value augmentation
    'degrees': 15.0,            # Rotation augmentation (+/- deg)
    'translate': 0.1,           # Translation augmentation (+/- fraction)
    'scale': 0.5,               # Scaling augmentation (+/- gain)
    'shear': 0.0,               # Shear augmentation (+/- deg)
    'perspective': 0.0,         # Perspective augmentation (+/- fraction)
    'flipud': 0.0,              # Flip up-down probability
    'fliplr': 0.5,              # Flip left-right probability
    'mosaic': 1.0,              # Mosaic augmentation probability
    'mixup': 0.0,               # Mixup augmentation probability
    'copy_paste': 0.0,          # Copy-paste augmentation probability
    'auto_augment': 'randaugment',  # Auto augmentation policy
    'erasing': 0.4,             # Random erasing probability
    'crop_fraction': 1.0,       # Image crop fraction
}

print("Starting YOLOv8-Nano training...")
print(f"Dataset: {YAML_PATH}")
print(f"Image size: 416x416")
print(f"Epochs: 100")

In [ ]:
# Start training
results = model.train(**training_params)

## 5. Evaluate Model Performance

In [ ]:
# Validate the model
metrics = model.val()

print("\n=== Validation Metrics ===")
print(f"mAP@50: {metrics.box.map50:.4f}")
print(f"mAP@50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

In [ ]:
# Plot training results
from IPython.display import Image, display

# Display results plot
results_path = f'{MODELS_PATH}/cattle_detector'
if os.path.exists(f'{results_path}/results.png'):
    display(Image(filename=f'{results_path}/results.png', width=800))

In [ ]:
# Test on sample images
test_images_path = f'{DATASET_PATH}/test/images'
test_images = os.listdir(test_images_path)[:5]

for img_name in test_images:
    img_path = os.path.join(test_images_path, img_name)
    results = model.predict(img_path, save=True, conf=0.25)
    
    # Display result
    result_img = f'{results[0].save_dir}/{img_name}'
    if os.path.exists(result_img):
        display(Image(filename=result_img, width=400))

## 6. Export to TFLite with INT8 Quantization

In [ ]:
# Export to TFLite format
# YOLOv8 supports direct TFLite export

# Load best model
best_model = YOLO(f'{MODELS_PATH}/cattle_detector/weights/best.pt')

# Export to TFLite
best_model.export(
    format='tflite',
    imgsz=416,
    int8=True,           # INT8 quantization
    data=YAML_PATH,      # Dataset for calibration
    batch=1,
    optimize=True,       # Optimize for mobile
    simplify=True,       # Simplify model
    opset=12,            # ONNX opset version
    workspace=4,         # TensorRT workspace size (GB)
)

print("Model exported to TFLite with INT8 quantization!")

In [ ]:
# Check exported model size
import glob

tflite_files = glob.glob(f'{MODELS_PATH}/cattle_detector/weights/*.tflite')
for tflite_file in tflite_files:
    size_mb = os.path.getsize(tflite_file) / (1024 * 1024)
    print(f"{os.path.basename(tflite_file)}: {size_mb:.2f} MB")

## 7. Test TFLite Model

In [ ]:
# Test TFLite model inference
import numpy as np
import tensorflow as tf

def test_tflite_model(tflite_path, test_image_path):
    """
    Test TFLite model inference on a single image.
    """
    # Load TFLite model
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    
    # Get input/output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    print(f"Input shape: {input_details[0]['shape']}")
    print(f"Input dtype: {input_details[0]['dtype']}")
    print(f"Output shape: {output_details[0]['shape']}")
    
    # Load and preprocess image
    img = cv2.imread(test_image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (416, 416))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    
    # Run inference
    import time
    start_time = time.time()
    
    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    
    inference_time = (time.time() - start_time) * 1000
    print(f"\nInference time: {inference_time:.2f} ms")
    print(f"Output shape: {output.shape}")
    
    return output

# Test on sample image
tflite_path = glob.glob(f'{MODELS_PATH}/cattle_detector/weights/*_int8.tflite')[0]
test_image = os.path.join(test_images_path, test_images[0])

print(f"Testing TFLite model: {tflite_path}")
print(f"Test image: {test_image}")

output = test_tflite_model(tflite_path, test_image)

## 8. Save Final Model

In [ ]:
# Copy final models to output directory
OUTPUT_DIR = f'{MODELS_PATH}/final'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Copy best PyTorch model
shutil.copy(
    f'{MODELS_PATH}/cattle_detector/weights/best.pt',
    f'{OUTPUT_DIR}/yolov8_nano_cattle_detector.pt'
)

# Copy TFLite model
for tflite_file in tflite_files:
    shutil.copy(tflite_file, OUTPUT_DIR)

print(f"Final models saved to: {OUTPUT_DIR}")
print("\nFiles:")
for f in os.listdir(OUTPUT_DIR):
    size_mb = os.path.getsize(os.path.join(OUTPUT_DIR, f)) / (1024 * 1024)
    print(f"  {f}: {size_mb:.2f} MB")

## 9. Model Summary

In [ ]:
print("="*60)
print("YOLOv8-Nano Detection Model Training Complete!")
print("="*60)
print(f"\nModel: YOLOv8-Nano")
print(f"Task: Cattle/Buffalo Detection")
print(f"Input Size: 416x416")
print(f"\nPerformance Metrics:")
print(f"  mAP@50: {metrics.box.map50:.4f}")
print(f"  mAP@50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")
print(f"\nModel Files:")
print(f"  PyTorch: yolov8_nano_cattle_detector.pt")
print(f"  TFLite INT8: *_int8.tflite (~5 MB)")
print(f"\nReady for Stage 2: Breed Classification with EfficientNet-B0")
print("="*60)